In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns

sns.set_style('ticks')

from tqdm import tqdm
from itertools import (count, product)
from scipy.stats import poisson

pool1_rent_rv = poisson(3)
pool1_return_rv = poisson(3)
pool2_rent_rv = poisson(4)
pool2_return_rv = poisson(2)

GAMMA = 0.9


In [2]:
def calc_reward(action, num_cars_rented):
    # action indexed 0 - 10 corresponding to -5 to 5 cars moved
    # where + indicates pool 1 to pool 2
    # 5 encodes no cards moved
    reward_from_action = -1 * np.abs(action - 5) * 2
    reward_from_cars = num_cars_rented * 10
    return reward_from_cars + reward_from_action

In [16]:
def calc_new_vals(cur_state_idx, values, action):
    # cur_state_idx -> num cars in each lot
    num_cars_pool_1 = cur_state_idx // 21
    num_cars_pool_2 = cur_state_idx % 21

    # incorporate action
    cars_moved = action - 5

    if cars_moved > 0 and cars_moved > num_cars_pool_1:
        cars_moved = num_cars_pool_1

    if cars_moved < 0 and np.abs(cars_moved) > num_cars_pool_2:
        cars_moved = -1 * num_cars_pool_2
    
    num_cars_pool_1 = np.clip(num_cars_pool_1 - cars_moved, 0, 20)
    num_cars_pool_2 = np.clip(num_cars_pool_2 + cars_moved, 0, 20)


    # sum over all values of next states and rewards
    # loop over possible next states
    val = 0

    for i in range(len(values)):
        pool_1 = i // 21
        pool_2 = i % 21

        diff_pool_1 = pool_1 - num_cars_pool_1
        diff_pool_2 = pool_2 - num_cars_pool_2

        min_car_rented_1 = max(0, -diff_pool_1)
        min_car_rented_2 = max(0, -diff_pool_2)

        for cars_rented_1, cars_rented_2 in product(range(int(min_car_rented_1), int(num_cars_pool_1) + 1), range(int(min_car_rented_2), int(num_cars_pool_2) + 1)):
            prob_cars_rented_1 = pool1_rent_rv.pmf(cars_rented_1)
            prob_cars_rented_2 = pool2_rent_rv.pmf(cars_rented_2)

            prob_cars_returned_1 = pool1_return_rv.pmf(diff_pool_1 + cars_rented_1)
            prob_cars_returned_2 = pool1_return_rv.pmf(diff_pool_1 + cars_rented_1)

            # reward
            reward = calc_reward(action, cars_rented_1 + cars_rented_2)

            val += (values[i] * GAMMA + reward) * prob_cars_rented_1 * prob_cars_rented_2 * prob_cars_returned_1 * prob_cars_returned_2
    
    return val




In [31]:
# policy evaluation

def policy_evaluation(policy, initial_values, threshold):
    # policy: numpy array num_states: vals as index of action, varies from 0 - 10
    # initial_values: numpy array num_states: vals as float indicating value
    # threshold: some small float
    # returns updated values
    delta = None
    values = initial_values
    while delta is None or delta > threshold:
        delta = 0.0
        # for each state
        for i, val in tqdm(enumerate(values)):
            action = policy[i]
            new_val = calc_new_vals(i, values, action)
            delta = max(np.abs(new_val - val), delta)
    return values


In [11]:
def policy_improvement(values):
    new_policy = np.zeros(len(values))
    val_maxes = np.zeros(len(values))
    for state in range(len(values)):
        val_max = None
        max_action = -1
        for action in range(11):
            val = calc_new_vals(state, values, action)
            if val_max is None or val > val_max:
                val_max = val
                max_action = action
        new_policy[state] = max_action
        val_maxes[state] = val_max
    return new_policy, val_maxes
            


In [12]:
def values_different(values, new_values, thres):
    return np.max(np.abs(new_values - values)) > thres

In [ ]:
# initialize
values = np.zeros(441)
policy = np.ones(441) * 5

thres = 0.1

new_policy, val_maxes = policy_improvement(values)

iteration = 0
while values_different(values, val_maxes, thres):
    iteration += 1
    print(f"evaluating policy {iteration}")
    policy = new_policy
    values = policy_evaluation(policy, values, thres)
    new_policy, val_maxes = policy_improvement(values)
    
